In [3]:
import numpy as np
import pandas as pd
import os
from datetime import datetime,timedelta
import math
import matplotlib.pyplot as plt
import seaborn  



In [4]:
from_str='2022-01-01'
to_str='2022-12-31'
feq ='Q'  

In [5]:
data_file='data/3CoreETFWorld.csv'
report_path='report'
title='3CoreETFWorld'



In [6]:
# build period datafrme
period_index=pd.date_range(start=from_str,end=to_str, freq=feq)
print(period_index)

DatetimeIndex(['2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31'], dtype='datetime64[ns]', freq='Q-DEC')


In [9]:
fundPerfByPeriod_df=pd.DataFrame(data=list(period_index),columns=['End_Date'])
fundPerfByPeriod_df['Start_Date']=np.nan
fundPerfByPeriod_df=fundPerfByPeriod_df[['Start_Date','End_Date']]
fundPerfByPeriod_df

,Start_Date,End_Date
0,NaN,2022-03-31
1,NaN,2022-06-30
2,NaN,2022-09-30
3,NaN,2022-12-31


In [11]:
print(f"Create the first date of each {feq} by iteration")
for index,row in fundPerfByPeriod_df.iterrows():
    if index==0:
       fundPerfByPeriod_df.iloc[0,0]=datetime.strptime(from_str,'%Y-%m-%d')
    else:
        # get End_Date(last day of Year, Quater,Month) of previouse row 
        prev_end_date= fundPerfByPeriod_df.iloc[index-1,1] 
        start_date=prev_end_date + timedelta(days = 1) # 
        fundPerfByPeriod_df.iloc[index,0]=start_date 
    # print(fundPerfByPeriod_df) 
    # print(f"=================index={index}=================")
print(fundPerfByPeriod_df.info())
fundPerfByPeriod_df

Create the first date of each Q by iteration
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Start_Date  4 non-null      object        
 1   End_Date    4 non-null      datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 192.0+ bytes
None


,Start_Date,End_Date
0,2022-01-01 00:00:00,2022-03-31
1,2022-04-01 00:00:00,2022-06-30
2,2022-07-01 00:00:00,2022-09-30
3,2022-10-01 00:00:00,2022-12-31


In [13]:
fundPerfByPeriod_df['Start_Date']=pd.to_datetime(fundPerfByPeriod_df['Start_Date'],format='%Y-%m-%d %H:%M:%S')  

fundPerfByPeriod_df['Period']= fundPerfByPeriod_df.apply( lambda item:f"{item['Start_Date'].strftime('%d')}{item['Start_Date'].strftime('%b')}{item['Start_Date'].strftime('%y')}-{item['End_Date'].strftime('%d')}{item['End_Date'].strftime('%b')}{item['End_Date'].strftime('%y')}"  ,axis=1  )
fundPerfByPeriod_df.info()
fundPerfByPeriod_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Start_Date  4 non-null      datetime64[ns]
 1   End_Date    4 non-null      datetime64[ns]
 2   Period      4 non-null      object        
dtypes: datetime64[ns](2), object(1)
memory usage: 224.0+ bytes


,Start_Date,End_Date,Period
0,2022-01-01,2022-03-31,01Jan22-31Mar22
1,2022-04-01,2022-06-30,01Apr22-30Jun22
2,2022-07-01,2022-09-30,01Jul22-30Sep22
3,2022-10-01,2022-12-31,01Oct22-31Dec22


In [14]:
df = pd.read_csv(data_file,index_col='Date/Time',parse_dates=['Date/Time'],dayfirst=True)
df.index.set_names('date',inplace=True)
df.rename(columns={'Ticker':'symbol','close':'price'},inplace=True)
df=df[['symbol','price']]
df.info()
print(df.tail(5))

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1602 entries, 2021-01-04 to 2023-02-15
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  1602 non-null   object 
 1   price   1602 non-null   float64
dtypes: float64(1), object(1)
memory usage: 37.5+ KB
           symbol   price
date                     
2023-02-09    QQQ  301.68
2023-02-10    QQQ  299.70
2023-02-13    QQQ  304.50
2023-02-14    QQQ  306.75
2023-02-15    QQQ  309.10


In [15]:
list_fund_name=df['symbol'].unique().tolist()  
print(list_fund_name)

['ACWI', 'SPY', 'QQQ']


In [16]:
dictPriceOfFund={}
for name in list_fund_name:
  fund_df=df.query('symbol==@name')
  fund_df=fund_df.loc[from_str:to_str,['price']]
  fund_df.sort_index(inplace=True)
  if len(fund_df)  >0:
      dictPriceOfFund[name]=fund_df  
      print(f"======={name} : {len(fund_df)}==========")  
      print(fund_df.tail(3))    
        

=======ACWI : 251==========
            price
date             
2022-12-28  83.93
2022-12-29  85.39
2022-12-30  84.88
=======SPY : 251==========
             price
date              
2022-12-28  376.66
2022-12-29  383.44
2022-12-30  382.43
=======QQQ : 251==========
             price
date              
2022-12-28  260.10
2022-12-29  266.44
2022-12-30  266.28
